<a href="https://colab.research.google.com/github/AbhayTyagi38/busbookingapp/blob/main/Multi_language_Translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install  transformers sentencepiece  gradio

In [ ]:
!pip install langdetect --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
from langdetect import detect
import gradio as gr

In [ ]:
model_name = "facebook/m2m100_418M"
tokenizer = M2M100Tokenizer.from_pretrained(model_name)
model = M2M100ForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

In [ ]:
language_name = {
    'en': 'English',
    'fr': 'French',
    'de': 'German',
    'es': 'Spanish',
    'hi': 'Hindi',
    'ru': 'Russian',
    'ja': 'Japanese',
    'zh': 'Chinese'
}

In [ ]:
def translate_text(text, tgt_lang):
    try:
        src_lang = detect(text)
        if src_lang not in language_name or tgt_lang not in language_name:
            return "Unsupported language."

        tokenizer.src_lang = src_lang
        encoded = tokenizer(text, return_tensors="pt")
        generated = model.generate(
            **encoded,
            forced_bos_token_id=tokenizer.get_lang_id(tgt_lang)
        )
        output = tokenizer.batch_decode(generated, skip_special_tokens=True)[0]

        return (
            f"Detected Language: {language_name.get(src_lang, src_lang)}\n"
            f"Translated ({language_name.get(tgt_lang)}): {output}"
        )
    except Exception as e:
        return f"Error: {str(e)}"

In [ ]:
lang_choices = list(language_name.keys())

gr.Interface(
    fn=translate_text,
    inputs=[
        gr.Textbox(label="Enter text to translate"),
        gr.Dropdown(choices=lang_choices, label="Target Language Code")
    ],
    outputs="text",
    title="🌍 AI Powered Multi language Translator",
    description="Auto-detects the input language and translates to the selected language using Meta's M2M100 model."
).launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0f62096d1b5e5bde61.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
